# 3D Taylor–Green — Classical vs NMSI–π*–γ_diss–e (Pseudo‑spectral Skeleton)

This notebook provides a minimal **3D incompressible NSE** pseudo‑spectral solver (periodic box)
to compare **classical** vs **augmented** (π*, γ_diss, e) runs on the Taylor–Green initial condition.

**Diagnostics:** Energy $E(t)$ and Enstrophy $\Omega(t)=\int |\omega|^2\,dx$ (here normalized as domain average).

⚠️ Educational skeleton — optimized for clarity, not for performance. Default grid is small to run on a laptop.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Grid
N = 32   # keep small for demo
L = 2*np.pi
x = np.linspace(0, L, N, endpoint=False)
y = np.linspace(0, L, N, endpoint=False)
z = np.linspace(0, L, N, endpoint=False)
X, Y, Z = np.meshgrid(x, y, z, indexing='ij')

# Spectral wavenumbers (i*k)
kx = 1j*2*np.pi*np.fft.fftfreq(N, d=L/(2*np.pi*N))
ky = 1j*2*np.pi*np.fft.fftfreq(N, d=L/(2*np.pi*N))
kz = 1j*2*np.pi*np.fft.fftfreq(N, d=L/(2*np.pi*N))
KX, KY, KZ = np.meshgrid(kx, ky, kz, indexing='ij')
k2 = KX**2 + KY**2 + KZ**2
k2[0,0,0] = 1.0

In [2]:
def project(u_hat, v_hat, w_hat):
    div_hat = KX*u_hat + KY*v_hat + KZ*w_hat
    u_hat -= KX*div_hat/k2
    v_hat -= KY*div_hat/k2
    w_hat -= KZ*div_hat/k2
    return u_hat, v_hat, w_hat

def grad(h):
    return KX*h, KY*h, KZ*h

def curl(u, v, w):
    u_hat, v_hat, w_hat = np.fft.fftn(u), np.fft.fftn(v), np.fft.fftn(w)
    ux, uy, uz = [np.fft.ifftn(g).real for g in grad(u_hat)]
    vx, vy, vz = [np.fft.ifftn(g).real for g in grad(v_hat)]
    wx, wy, wz = [np.fft.ifftn(g).real for g in grad(w_hat)]
    wx_v = wz*vy - wy*vz  # dummy to keep structure (not used)
    # vorticity components (ω = ∇ × u)
    wx = wy = wz = None
    wx = np.fft.ifftn(KY*w_hat - KZ*v_hat).real
    wy = np.fft.ifftn(KZ*u_hat - KX*w_hat).real
    wz = np.fft.ifftn(KX*v_hat - KY*u_hat).real
    return wx, wy, wz

def energy(u,v,w):
    return 0.5*np.mean(u*u + v*v + w*w)

def enstrophy(u,v,w):
    wx, wy, wz = curl(u,v,w)
    return np.mean(wx*wx + wy*wy + wz*wz)

def nonlinear(u,v,w):
    u_hat, v_hat, w_hat = np.fft.fftn(u), np.fft.fftn(v), np.fft.fftn(w)
    ux, uy, uz = [np.fft.ifftn(g).real for g in grad(u_hat)]
    vx, vy, vz = [np.fft.ifftn(g).real for g in grad(v_hat)]
    wx, wy, wz = [np.fft.ifftn(g).real for g in grad(w_hat)]
    Nu = -(u*ux + v*uy + w*uz)
    Nv = -(u*vx + v*vy + w*vz)
    Nw = -(u*wx + v*wy + w*wz)
    Nu_hat, Nv_hat, Nw_hat = np.fft.fftn(Nu), np.fft.fftn(Nv), np.fft.fftn(Nw)
    return project(Nu_hat, Nv_hat, Nw_hat)

In [3]:
def init_TG3D(A=1.0):
    # Standard 3D Taylor–Green vortex (k=1)
    u0 =  A*np.sin(X)*np.cos(Y)*np.cos(Z)
    v0 = -A*np.cos(X)*np.sin(Y)*np.cos(Z)
    w0 =  np.zeros_like(X)
    # project to div-free
    u_hat, v_hat, w_hat = project(np.fft.fftn(u0), np.fft.fftn(v0), np.fft.fftn(w0))
    return [np.fft.ifftn(h).real for h in (u_hat, v_hat, w_hat)]

def step_SSPRK3_3d(u,v,w, rhs, dt):
    # k1
    ru1, rv1, rw1 = [np.fft.ifftn(h).real for h in rhs(u,v,w)]
    u1, v1, w1 = u + dt*ru1, v + dt*rv1, w + dt*rw1
    # k2
    ru2, rv2, rw2 = [np.fft.ifftn(h).real for h in rhs(u1,v1,w1)]
    u2 = 0.75*u + 0.25*(u1 + dt*ru2)
    v2 = 0.75*v + 0.25*(v1 + dt*rv2)
    w2 = 0.75*w + 0.25*(w1 + dt*rw2)
    # k3
    ru3, rv3, rw3 = [np.fft.ifftn(h).real for h in rhs(u2,v2,w2)]
    un = (1/3)*u + (2/3)*(u2 + dt*ru3)
    vn = (1/3)*v + (2/3)*(v2 + dt*rv3)
    wn = (1/3)*w + (2/3)*(w2 + dt*rw3)
    return un, vn, wn

def make_rhs3d(nu=5e-4, A_pi=0.0, omega_pi=3.5, zeta=0.0, z_thresh=np.inf,
               lam_e=0.0, alpha_e=0.0):
    def rhs(u,v,w, t=[0.0]):
        t0 = t[0]
        Nu_hat, Nv_hat, Nw_hat = nonlinear(u,v,w)
        u_hat, v_hat, w_hat = np.fft.fftn(u), np.fft.fftn(v), np.fft.fftn(w)
        visc_u = -nu*k2*u_hat
        visc_v = -nu*k2*v_hat
        visc_w = -nu*k2*w_hat

        aug = 0.0
        if A_pi != 0.0:
            aug += A_pi*np.sin(omega_pi*t0)
        if zeta != 0.0:
            if enstrophy(u,v,w) > z_thresh:
                aug += -zeta
        if lam_e != 0.0:
            aug += -lam_e*np.exp(-alpha_e*t0)

        Ru = Nu_hat + visc_u + aug*u_hat
        Rv = Nv_hat + visc_v + aug*v_hat
        Rw = Nw_hat + visc_w + aug*w_hat
        return project(Ru, Rv, Rw)
    return rhs

In [4]:
# Driver
def run_3d(T=2.0, dt=1e-3, nu=5e-4, augmented=False,
           A_pi=0.15, omega_pi=3.5, zeta=0.8, z_thresh=0.05,
           lam_e=0.6, alpha_e=0.25, A0=1.0):
    u,v,w = init_TG3D(A=A0)
    t=0.0
    E_hist, Z_hist, T_hist = [], [], []
    nsteps = int(T/dt)
    for n in range(nsteps):
        rhs = make_rhs3d(nu=nu,
                         A_pi=(A_pi if augmented else 0.0), omega_pi=omega_pi,
                         zeta=(zeta if augmented else 0.0), z_thresh=z_thresh,
                         lam_e=(lam_e if augmented else 0.0), alpha_e=alpha_e)
        rhs.__defaults__[0][0] = t
        u,v,w = step_SSPRK3_3d(u,v,w, rhs, dt)
        t += dt
        if n % 10 == 0:
            E_hist.append(energy(u,v,w))
            Z_hist.append(enstrophy(u,v,w))
            T_hist.append(t)
    return np.array(T_hist), np.array(E_hist), np.array(Z_hist)

In [5]:
# Compare classical vs augmented (small demo)
T1,E1,Z1 = run_3d(augmented=False)
T2,E2,Z2 = run_3d(augmented=True)

fig, ax = plt.subplots(1,2, figsize=(11,4.2))
ax[0].plot(T1,E1,label='Classical'); ax[0].plot(T2,E2,label='Augmented')
ax[0].set_title('Energy E(t)'); ax[0].set_xlabel('t'); ax[0].set_ylabel('E'); ax[0].grid(True); ax[0].legend()
ax[1].plot(T1,Z1,label='Classical'); ax[1].plot(T2,Z2,label='Augmented')
ax[1].set_title('Enstrophy Ω(t)'); ax[1].set_xlabel('t'); ax[1].set_ylabel('Ω'); ax[1].grid(True); ax[1].legend()
plt.tight_layout(); plt.show()
np.savez('out_3d_tg_compare.npz', T1=T1, E1=E1, Z1=Z1, T2=T2, E2=E2, Z2=Z2)